In [1]:
!pip install datasets evaluate gradio transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.3 MB/s eta 0:00:00


In [2]:
import os
os.environ["WANDB_DISABLED"] = "true"  # no API key requests

import torch
import numpy as np
from datasets import load_dataset
from transformers import (AutoTokenizer, AutoModelForSequenceClassification,
                          DataCollatorWithPadding, TrainingArguments, Trainer)
import evaluate


In [3]:
dataset = load_dataset("ag_news")
dataset["train"] = dataset["train"].select(range(20))  # only 20 samples for training
dataset["test"] = dataset["test"].select(range(5))     # only 5 samples for testing


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [4]:
model_name = "prajjwal1/bert-tiny"  # smallest BERT
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_examples(examples):
    return tokenizer(examples["text"], truncation=True, max_length=32)

tokenized = dataset.map(preprocess_examples, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [5]:
num_labels = 4  # AG News: World, Sports, Business, Sci/Tech
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)


pytorch_model.bin:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "f1_weighted": f1.compute(predictions=preds, references=labels, average="weighted")["f1"]
    }


In [7]:
training_args = TrainingArguments(
    output_dir="./tiny-bert-model",
    do_eval=True,
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    logging_steps=1,
    learning_rate=5e-5,
    fp16=torch.cuda.is_available()
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [8]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


/tmp/ipython-input-951079361.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [9]:
trainer.train()


Step,Training Loss
1,1.355800
2,1.378400
3,1.254100
4,1.292200
5,1.239300


TrainOutput(global_step=5, training_loss=1.3039635181427003, metrics={'train_runtime': 1.6991, 'train_samples_per_second': 11.771, 'train_steps_per_second': 2.943, 'total_flos': 1589099520.0, 'train_loss': 1.3039635181427003, 'epoch': 1.0})

In [10]:
metrics = trainer.evaluate()
print(metrics)


{'eval_loss': 1.438141107559204, 'eval_accuracy': 0.2, 'eval_f1_weighted': 0.08, 'eval_runtime': 0.1306, 'eval_samples_per_second': 38.284, 'eval_steps_per_second': 15.313, 'epoch': 1.0}


In [11]:
from transformers import pipeline

label_map = {0: "World", 1: "Sports", 2: "Business", 3: "Sci/Tech"}
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer)

def predict_single(text):
    out = pipe(text)[0]
    label_str = out['label']
    if label_str.startswith("LABEL_"):
        idx = int(label_str.split("_")[-1])
        label_str = label_map.get(idx, label_str)
    return f"{label_str} (score: {out['score']:.3f})"

print(predict_single("NASA announces a new Mars rover mission"))


Device set to use cpu


Sports (score: 0.265)


In [12]:
import gradio as gr

def gradio_predict(text):
    return predict_single(text)

demo = gr.Interface(
    fn=gradio_predict,
    inputs=gr.Textbox(lines=3, placeholder="Enter news headline..."),
    outputs="text",
    title="Tiny AG News Classifier (BERT-Tiny)",
    description="Classifies news headlines into World, Sports, Business, or Sci/Tech"
)

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2a1fedb5a2378c9750.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
